Transplant train/validate/test model from Hex to VS Code

In [60]:
# ============================================
# Environment setup (local execution)
# ============================================

import numpy as np
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import joblib
import json

In [61]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 200)

In [62]:
# Only need train data for this workflow
train_data = pd.read_csv("../data/raw/train.csv")

# Quick sanity checks
print("Rows, cols:", train_data.shape)
train_data.head()

Rows, cols: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [63]:
y = train_data['Survived']
X = train_data.drop(columns=['Survived'])

In [64]:
# Define a function to perform the data transformations
def transform_data(data):
    data = data.copy()
    
    # Drop 'Name' column
    data.drop(columns=['Name'], inplace=True)
    
    # Convert 'Sex' to 0 for 'male' and 1 for 'female'
    sex_mapping = {'male': 0, 'female': 1}
    data['Sex'] = data['Sex'].map(sex_mapping)

    # Encode Age as a binary "present vs missing" indicator (unchanged logic)
    data['Age'] = data['Age'].fillna(0)
    data['Age'] = data['Age'].apply(lambda x: 1 if x > 0 else x)
    
    # Rename for correct semantics (NO value change)
    data.rename(columns={'Age': 'AgeMissing'}, inplace=True)

    # Create a new feature 'FamSz' by summing 'SibSp' and 'Parch'
    data['FamSz'] = data['SibSp'] + data['Parch']

    # Drop 'Ticket', 'Cabin', 'Fare', 'PassengerId' columns
    data.drop(columns=['Ticket', 'Cabin', 'Fare', 'PassengerId'], inplace=True)

    # Fill missing values in 'Embarked' with mode
    embarked_mode = data['Embarked'].mode()[0]
    data['Embarked'] = data['Embarked'].fillna(embarked_mode)

    # Convert 'Embarked' to numeric values (1, 2, 3)
    embarked_mapping = {'S': 1, 'C': 2, 'Q': 3}
    data['Embarked'] = data['Embarked'].map(embarked_mapping)
    
    return data

In [65]:
y = train_data["Survived"]
X = transform_data(train_data.drop(columns=["Survived"]))

In [66]:
feature_names = X.columns.tolist()
print(feature_names)

with open("../artifacts/feature_names.json", "w") as f:
    json.dump(feature_names, f, indent=2)

['Pclass', 'Sex', 'AgeMissing', 'SibSp', 'Parch', 'Embarked', 'FamSz']


In [67]:
print("X shape:", X.shape)
print("Missing values:", X.isnull().sum().sum())
X.head()

X shape: (891, 7)
Missing values: 0


,Pclass,Sex,AgeMissing,SibSp,Parch,Embarked,FamSz
0,3,0,1.0,1,0,1,1
1,1,1,1.0,1,0,2,1
2,3,1,1.0,0,0,1,0
3,1,1,1.0,1,0,1,1
4,3,0,1.0,0,0,1,0


In [68]:
from sklearn.model_selection import StratifiedKFold

RANDOM_STATE = 42
cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("rf", RandomForestClassifier(random_state=42))
    ]
)

param_grid = {
    "rf__n_estimators": [2, 4, 8, 10, 25, 50, 100],
    "rf__max_depth": [None, 2, 3, 4, 8, 16],
    "rf__min_samples_split": [2, 5, 10],
    "rf__min_samples_leaf": [1, 2, 4, 6],
    "rf__bootstrap": [True, False],
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv10,
    n_jobs=-1,
    verbose=2
)

In [70]:
# ============================================
# Run GridSearchCV
# ============================================

grid_search.fit(X, y)

Fitting 10 folds for each of 1008 candidates, totalling 10080 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'rf__bootstrap': [True, False], 'rf__max_depth': [None, 2, ...], 'rf__min_samples_leaf': [1, 2, ...], 'rf__min_samples_split': [2, 5, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1

In [71]:
# ============================================
# Inspect GridSearch results
# ============================================

print("Best CV score:", grid_search.best_score_)
print("Best parameters:")
for k, v in grid_search.best_params_.items():
    print(f"  {k}: {v}")

Best CV score: 0.8136454431960051
Best parameters:
  rf__bootstrap: True
  rf__max_depth: 3
  rf__min_samples_leaf: 1
  rf__min_samples_split: 2
  rf__n_estimators: 25


In [72]:
# ============================================
# Final hyperparameters (OOF-selected)
# ============================================

FINAL_RF_PARAMS = {
    "bootstrap": True,
    "max_depth": 3,
    "min_samples_leaf": 1,
    "min_samples_split": 2,
    "n_estimators": 25,
    "random_state": 42
}

FINAL_RF_PARAMS

{'bootstrap': True,
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 25,
 'random_state': 42}

In [73]:
# ============================================
# Final scaler fit (100% of data)
# ============================================

from sklearn.preprocessing import StandardScaler

final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(X)

# Wrap back into DataFrame to preserve feature names
X_scaled_df = pd.DataFrame(
    X_scaled,
    columns=X.columns,
    index=X.index
)

X_scaled_df.head()

,Pclass,Sex,AgeMissing,SibSp,Parch,Embarked,FamSz
0,0.827377,-0.737695,0.497895,0.432793,-0.473674,-0.568837,0.059160
1,-1.566107,1.355574,0.497895,0.432793,-0.473674,1.005181,0.059160
2,0.827377,1.355574,0.497895,-0.474545,-0.473674,-0.568837,-0.560975
3,-1.566107,1.355574,0.497895,0.432793,-0.473674,-0.568837,0.059160
4,0.827377,-0.737695,0.497895,-0.474545,-0.473674,-0.568837,-0.560975


In [74]:
# ============================================
# Final model fit (100% of data)
# ============================================

from sklearn.ensemble import RandomForestClassifier

final_model = RandomForestClassifier(**FINAL_RF_PARAMS)
final_model.fit(X_scaled_df, y)

final_model

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",25
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(y_t

In [75]:
# ============================================
# Export artifacts for Hex
# ============================================

import joblib
import json
import os

os.makedirs("artifacts", exist_ok=True)

# Model + scaler
joblib.dump(final_model, "../artifacts/model.joblib")
joblib.dump(final_scaler, "../artifacts/scaler.joblib")

# Feature names (contract)
with open("../artifacts/feature_names.json", "w") as f:
    json.dump(list(X.columns), f, indent=2)

print("Artifacts written:")
print(" - artifacts/model.joblib")
print(" - artifacts/scaler.joblib")
print(" - artifacts/feature_names.json")

Artifacts written:
 - artifacts/model.joblib
 - artifacts/scaler.joblib
 - artifacts/feature_names.json


In [76]:
# ============================================
# Sanity check: reload artifacts
# ============================================

loaded_model = joblib.load("../artifacts/model.joblib")
loaded_scaler = joblib.load("../artifacts/scaler.joblib")

# Pick a few rows
X_sample = X.iloc[:5]
X_sample_scaled = pd.DataFrame(
    loaded_scaler.transform(X_sample),
    columns=X_sample.columns,
    index=X_sample.index
)

probs = loaded_model.predict_proba(X_sample_scaled)[:, 1]
probs

array([0.26465134, 0.83295516, 0.48551084, 0.8162008 , 0.18669138])